# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [1]:
import numpy as np
from scipy import signal

from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer

# add the Contrib dir that contains all tools developped by MB : mbTools.py
#sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'python'))
#print(os.path.join(os.path.dirname(sys.path[0]),'python'))
import mbTools

### Import local config, create it if inexistant
All user-specific configuration (projects, defautl path to files...) are stored in a file named localConfig.ini in the python subfolder of AudreyHayLab repo. It is ignored by git so that it remains truely local. If the file does not exist at beginning, it is created with default values that can be modified at whishes.

> ℹ️ **Not in used for AB**

In [2]:
config = mbTools.localConf()
#rawDataPath = config['DATA']['path']
#print(f'All raw data are expected to be found in the folder: {rawDataPath}')
#analysisPath = config['ANALYSIS']['path']
#print(f'All analysis will be saved in the folder: {analysisPath}')
#config.printAll()

Local config file loaded from localConfig.ini


### Choose experiment

In [3]:
dpath = None
%store -r dpath
print(dpath)
try:
    theExpe = mbTools.experiment(dpath)
except:
    theExpe = mbTools.experiment()

/Volumes/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/RedLinesOK/session1/OpenEphys/


FileChooser(path='/Volumes/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/…

## Load LFPs data

In [4]:
All = theExpe.loadLFP()
All.shape

NPY file loaded, with 31 channels and 940555 datapoint


(940555, 31)

In [ ]:
#BOB
theExpe.defineMap('EMG',[dict(canal = 27, status=1)])
theExpe.defineMap('S1',[dict(canal = 5, status=1),
                         dict(canal = 6, status=2)])
theExpe.defineMap('PFC',[dict(canal = 2, status=1),
                         dict(canal = 3, status=2)])
theExpe.defineMap('CA1',[dict(canal = 29, status=1),
                         dict(canal = 30, status=2)])
theExpe.combineStructures(theExpe.channelsMap)

In [ ]:
# Filtre parameter:
f_lowcut = 1
f_hicut = 50.
fs = 1000
nyq = 0.5 * fs
N = 4                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')


In [ ]:
#Blue
PFC=All[:, 14]-All[:,15]
PFC = signal.filtfilt(b, a, PFC)
S1=All[:, 8]-All[:,9]
S1 = signal.filtfilt(b, a, S1)
EMG=All[:,20]
combined = np.stack([S1,PFC, EMG/10], axis = 1)

In [ ]:
#Pink
PFC=All[:, 26+32]-All[:,27+32]
EMG=All[:,9+32]
S1=All[:, 0+32]-All[:,1+32]
combined = np.stack([S1,PFC, EMG/10], axis = 1)

### Visualise All

In [8]:
%gui qt
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')
#add them to mainwindow

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

win.add_view(view1)


#Run
win.show()


In [9]:
%gui qt
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
#view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')

source = InMemoryAnalogSignalSource(combined, sample_rate, t_start)
view1 = TraceViewer(source=source, name='trace')

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

#create a time freq viewer conencted to the same source
view2 = TimeFreqViewer(source=source, name='tfr')

view2.params['show_axis'] = True
view2.params['timefreq', 'deltafreq'] = 1
view2.params['timefreq', 'f_stop'] = 200
#view2.by_channel_params['ch0', 'timefreq', 'f_stop'] = 20
#view2.by_channel_params['ch1', 'timefreq', 'f_stop'] = 150
view2.by_channel_params['ch0', 'visible'] = False
view2.by_channel_params['ch1', 'visible'] = True
view2.by_channel_params['ch1', 'clim'] = 100

#add them to mainwindow
win.add_view(view1)
win.add_view(view2)


#Run
win.show()


: 